In [1]:
!{'gcloud config set project poc-dbs'}
!{'pip install -r ../../requirements.txt'}

Updated property [core/project].


In [2]:
import apache_beam as beam
import logging
import json
import numpy as np

DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'

In [3]:
def compute_stats(airport, events):
    arrived = [event['ARR_DELAY'] for event in events if event['EVENT_TYPE'] == 'arrived']
    avg_arr_delay = float(np.mean(arrived)) if len(arrived) > 0 else None

    departed = [event['DEP_DELAY'] for event in events if event['EVENT_TYPE'] == 'departed']
    avg_dep_delay = float(np.mean(departed)) if len(departed) > 0 else None

    num_flights = len(events)
    start_time = min([event['EVENT_TIME'] for event in events])
    latest_time = max([event['EVENT_TIME'] for event in events])

    return {
        'AIRPORT': airport,
        'AVG_ARR_DELAY': avg_arr_delay,
        'AVG_DEP_DELAY': avg_dep_delay,
        'NUM_FLIGHTS': num_flights,
        'START_TIME': start_time,
        'END_TIME': latest_time
    }


def by_airport(event):
    if event['EVENT_TYPE'] == 'departed':
        return event['ORIGIN'], event
    else:
        return event['DEST'], event

project='poc-dbs'
bucket='poc-dbs-dsongcp'
region='europe-west4'

argv = [
    '--project={0}'.format(project),
    '--job_name=ch04avgdelay',
    '--streaming',
    '--save_main_session',
    '--staging_location=gs://{0}/flights/staging/'.format(bucket),
    '--temp_location=gs://{0}/flights/temp/'.format(bucket),
    '--autoscaling_algorithm=THROUGHPUT_BASED',
    '--max_num_workers=8',
    '--region={}'.format(region),
    '--runner=DirectRunner'
]

In [4]:
!{'gcloud config set project poc-dbs'}
!{'export GOOGLE_CLOUD_PROJECT=poc-dbs'}
with beam.Pipeline(argv=argv) as pipeline:
        events = {}

        for event_name in ['arrived', 'departed']:
            topic_name = "projects/{}/topics/{}".format(project, event_name)

            events[event_name] = (pipeline
                                  | 'read:{}'.format(event_name) >> beam.io.ReadFromPubSub(
                                                topic=topic_name)
                                  | 'parse:{}'.format(event_name) >> beam.Map(lambda s: json.loads(s))
                                  )

        all_events = (events['arrived'], events['departed']) | beam.Flatten()

        stats = (all_events
                 | 'byairport' >> beam.Map(by_airport)
                 | 'window' >> beam.WindowInto(beam.window.SlidingWindows(60 * 60, 5 * 60))
                 | 'group' >> beam.GroupByKey()
                 | 'stats' >> beam.Map(lambda x: compute_stats(x[0], x[1]))
                 | beam.Map(lambda x: print(x))
        )


        """
        stats_schema = ','.join(['AIRPORT:string,AVG_ARR_DELAY:float,AVG_DEP_DELAY:float',
                                 'NUM_FLIGHTS:int64,START_TIME:timestamp,END_TIME:timestamp']
                                 )
        (stats
         | 'bqout' >> beam.io.gcp.bigquery.WriteToBigQuery(
                    table='dsongcp.streaming_delays', 
                    schema=stats_schema
                )
         )"""

Updated property [core/project].


KeyboardInterrupt: 

ERROR:apache_beam.runners.common:unsupported operand type(s) for /: 'NoneType' and 'int' [while running '[4]: stats']
Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1434, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 637, in apache_beam.runners.common.SimpleInvoker.invoke_process
  File "/home/antoine_baret/data-science-on-gcp/.venv/lib/python3.9/site-packages/apache_beam/transforms/core.py", line 1963, in <lambda>
    wrapper = lambda x: [fn(x)]
  File "/tmp/ipykernel_3858/2327646342.py", line 21, in <lambda>
    | 'stats' >> beam.Map(lambda x: compute_stats(x[0], x[1]))
  File "/tmp/ipykernel_3858/3647296015.py", line 3, in compute_stats
    avg_arr_delay = float(np.mean(arrived)) if len(arrived) > 0 else None
  File "<__array_function__ internals>", line 200, in mean
  File "/home/antoine_baret/data-science-on-gcp/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py", line 3464, in mean
    retur